# Use Gen AI Evaluation SDK to Evaluate Models in Vertex AI Studio

This notebook demonstrates how to get started with using the *Vertex AI Python SDK for Gen AI Evaluation Service* for generative models in Vertex AI Studio, Model Garden, and Model Registry.

Gen AI Evaluation Service empowers you to comprehensively assess and enhance your generative AI models and applications. Whether you're selecting the ideal model, optimizing prompt templates, or evaluating fine-tuned checkpoints, this service provides the tools and insights you need.

In this Colab tutorial, we'll explore three major use cases:

1.  Run Evaluation on 1P Models
  *   Learn how to evaluate `Gemini` models in Vertex AI Studio using the *Gen AI Evaluation Service SDK*.

  *   Explore different evaluation metrics and techniques for assessing performance on various tasks.

  *   Discover how to leverage the SDK for in-depth analysis and comparison of `Gemini` model variants.
2.  Prompt Engineering

  *   Explore the impact of prompt design on model performance.
  *   Utilize the SDK to systematically evaluate and refine your prompts.


For additional use cases and advanced features, refer to our public documentation and notebook tutorials for evaluation use cases:

* https://cloud.google.com/vertex-ai/generative-ai/docs/models/evaluation-overview#notebooks_for_evaluation_use_cases

* https://cloud.google.com/vertex-ai/generative-ai/docs/models/run-evaluation

Let's get started!

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10

## Getting Started

### Install Vertex AI SDK for Gen AI Evaluation Service

In [ ]:
%pip install -U -q google-cloud-aiplatform[evaluation] openai

### Authenticate your notebook environment (Colab only)

In [ ]:
import sys

if "google.colab" in sys.modules:
  from google.colab import auth

  auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

In [ ]:
PROJECT_ID = "project-ee7cb7bb-2d31-45e1-83d"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
  raise ValueError("Please set your PROJECT_ID")


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [ ]:
import openai
import pandas as pd

## Load an evaluation dataset

Load a subset of the `OpenOrca` dataset using the `huggingface/datasets` library. We will use 10 samples from the first 100 rows of the "train" split of `OpenOrca` dataset to demonstrate evaluating prompts and model responses in this Colab.

### Dataset Summary

The OpenOrca dataset is a collection of augmented [FLAN Collection data](https://arxiv.org/abs/2301.13688). Currently ~1M GPT-4 completions, and ~3.2M GPT-3.5 completions. It is tabularized in alignment with the distributions presented in the ORCA paper and currently represents a partial completion of the full intended dataset, with ongoing generation to expand its scope. The data is primarily used for training and evaluation in the field of natural language processing.


In [ ]:
from datasets import load_dataset
from google.auth import default, transport
from vertexai.evaluation import (
    EvalTask,
    MetricPromptTemplateExamples,
    PairwiseMetric,
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
)
from vertexai.generative_models import GenerativeModel
from vertexai.preview.evaluation import notebook_utils

ds = (
    load_dataset(
        "Open-Orca/OpenOrca",
        data_files="1M-GPT4-Augmented.parquet",
        split="train[:100]",
    )
    .to_pandas()
    .drop(columns=["id"])
    .rename(columns={"response": "reference"})
)

dataset = ds.sample(n=10)

#### Preview the dataset

In [ ]:
dataset.head()

## Run evaluation on Gemini models

The *Gen AI Evaluation Service SDK* includes native support for evaluating Gemini models. This streamlines the evaluation process for Google's latest and most capable family of large language models. With minimal coding effort, you can leverage pre-defined metrics and workflows to assess the performance of Gemini models on various tasks. You can also customize your own model-based metrics based on your specific evaluation criteria.

This enhanced support enables you to:

- **Quickly evaluate Gemini models:** Effortlessly assess the performance of Gemini models using the SDK's streamlined workflows.
- **Compare models side-by-side:** Benchmark Gemini against other models to understand relative strengths and weaknesses.
- **Analyze prompt templates:** Evaluate the effectiveness of different prompt designs for optimizing Gemini's performance.

This native integration simplifies the evaluation process, allowing you to focus on understanding and improving the capabilities of Gemini.

#### Understand the `EvalTask` class

The `EvalTask` class is a core component of the *Gen AI Evaluation Service SDK* framework. It allows you to define and run evaluation jobs against your Gen AI models/applications, providing a structured way to measure their performance on specific tasks. Think of an `EvalTask` as a blueprint for your evaluation process.

`EvalTask` class requires an evaluation dataset and a list of metrics. Supported metrics are documented on the Generative AI on Vertex AI [Define your evaluation metrics](https://cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval) page. The dataset can be an `pandas.DataFrame`, Python dictionary or a file path URI and can contain default column names such as `prompt`, `reference`, `response`, and `baseline_model_response`.

* **Bring-your-own-response (BYOR):** You already have the data that you want to evaluate stored in the dataset. You can customize the response column names for both your model and the baseline model using parameters like `response_column_name` and `baseline_model_response_column_name` or through the `metric_column_mapping`.

* **Perform model inference without a prompt template:** You have a dataset containing the input prompts to the model and want to perform model inference before evaluation. A column named `prompt` is required in the evaluation dataset and is used directly as input to the model.

* **Perform model inference with a prompt template:** You have a dataset containing the input variables to the prompt template and want to assemble the prompts for model inference. Evaluation dataset must contain column names corresponding to the variable names in the prompt template. For example, if prompt template is "Instruction: {instruction}, context: {context}", the dataset must contain `instruction` and `context` columns.



`EvalTask` supports extensive evaluation scenarios including BYOR, model inference with Gemini models, 3P models endpoints/SDK clients, or custom model generation functions, using computation-based metrics, model-based pointwise and pairwise metrics. The `evaluate()` method triggers the evaluation process, optionally taking a model, prompt template, experiment logging configurations, and other evaluation run configurations. You can view the SDK reference documentation for [Gen AI Evaluation package](https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.evaluation) for more details.

### Define a model

In [ ]:
# Model to be evaluated
model = GenerativeModel(
    "gemini-2.0-flash",
    generation_config={
        "temperature": 0.6,
        "max_output_tokens": 256,
        "top_k": 1,
    },
)

### Use computation-based metrics


In [ ]:
# Define an EvalTask with ROUGE-L-SUM metric
rouge_eval_task = EvalTask(
    dataset=dataset,
    metrics=["rouge_l_sum"],
)
rouge_result = rouge_eval_task.evaluate(
    model=model,
    prompt_template="# System_prompt\n{system_prompt} # Question\n{question}",
)

In [ ]:
notebook_utils.display_eval_result(rouge_result)

### Use model-based pointwise metrics

In [ ]:
# @title Select a pointwise metric to use

import ipywidgets as widgets

pointwise_single_turn_metrics = [
    metric
    for metric in MetricPromptTemplateExamples.list_example_metric_names()
    if not metric.startswith("pairwise") and not metric.startswith("multi_turn")
]

dropdown = widgets.Dropdown(
    options=pointwise_single_turn_metrics,
    description="Select a metric:",
    font_weight="bold",
    style={"description_width": "initial"},
)


def dropdown_eventhandler(change):
  global POINTWISE_METRIC
  if change["type"] == "change" and change["name"] == "value":
    POINTWISE_METRIC = change.new
    print("Selected:", change.new)


POINTWISE_METRIC = dropdown.value
dropdown.observe(dropdown_eventhandler, names="value")
display(dropdown)

In [ ]:
pointwise_result = EvalTask(
    dataset=dataset,
    metrics=[POINTWISE_METRIC],
).evaluate(
    model=model,
    prompt_template="# System_prompt\n{system_prompt} # Question\n{question}",
)

In [ ]:
notebook_utils.display_eval_result(pointwise_result)

In [ ]:
notebook_utils.display_explanations(
    pointwise_result, num=1, metrics=[POINTWISE_METRIC]
)

#### Build your own pointwise metric

For more information about metric customization, see this [notebook tutorial](https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/customize_model_based_metrics.ipynb).

In [ ]:
pointwise_result = EvalTask(
    dataset=dataset,
    metrics=[
        # Create a unique model-based metric for your own use cases
        PointwiseMetric(
            metric="linguistic_acceptability",
            metric_prompt_template=PointwiseMetricPromptTemplate(
                criteria={
                    "Proper Grammar": (
                        "The language's grammar rules are correctly followed,"
                        " including but not limited to sentence structures,"
                        " verb tenses, subject-verb agreement, proper"
                        " punctuation, and capitalization."
                    ),
                    "Appropriate word choice": (
                        "Words chosen are appropriate and purposeful given"
                        " their relative context and positioning in the text."
                        " Vocabulary demonstrates prompt understanding."
                    ),
                    "Reference Alignment": (
                        "The response is consistent and aligned with the"
                        " reference."
                    ),
                },
                rating_rubric={
                    "5": (
                        "Excellent: The writing is grammatically correct, uses"
                        " appropriate vocabulary and aligns perfectly with the"
                        " reference."
                    ),
                    "4": (
                        "Good: The writing is generally grammatically correct,"
                        " uses appropriate vocabulary and aligns well with the"
                        " reference."
                    ),
                    "3": (
                        "Satisfactory: The writing may have minor grammatical"
                        " errors or use less-appropriate vocabulary, but it"
                        " aligns reasonably well with the reference."
                    ),
                    "2": (
                        "Unsatisfactory: The writing has significant"
                        " grammatical errors, uses inappropriate vocabulary,"
                        " deviates significantly from the reference."
                    ),
                    "1": (
                        "Poor: The writing is riddled with grammatical errors,"
                        " uses highly inappropriate vocabulary, is completely"
                        " unrelated to the reference."
                    ),
                },
                input_variables=["prompt", "reference"],
            ),
        )
    ],
).evaluate(
    model=model,
    prompt_template="# System_prompt\n{system_prompt} # Question\n{question}",
)

In [ ]:
notebook_utils.display_eval_result(pointwise_result)

### Use model-based pairwise metrics

Evaluate two Gen AI models side-by-side (SxS) with model-based pairwise metrics.

In [ ]:
# @title Select a pairwise metric to use

from IPython.display import display
import ipywidgets as widgets

pairwise_single_turn_metrics = [
    metric
    for metric in MetricPromptTemplateExamples.list_example_metric_names()
    if metric.startswith("pairwise") and "multi_turn" not in metric
]

dropdown = widgets.Dropdown(
    options=pairwise_single_turn_metrics,
    description="Select a metric:",
    font_weight="bold",
    style={"description_width": "initial"},
)


def dropdown_eventhandler(change):
  global PAIRWISE_METRIC_NAME
  if change["type"] == "change" and change["name"] == "value":
    PAIRWISE_METRIC_NAME = change.new
    print("Selected:", change.new)


PAIRWISE_METRIC_NAME = dropdown.value
dropdown.observe(dropdown_eventhandler, names="value")
display(dropdown)

#### Run SxS evaluation

In [ ]:
pairwise_result = EvalTask(
    dataset=dataset,
    metrics=[
        PairwiseMetric(
            metric=PAIRWISE_METRIC_NAME,
            metric_prompt_template=MetricPromptTemplateExamples.get_prompt_template(
                PAIRWISE_METRIC_NAME
            ),
            # Define a baseline model to compare against
            baseline_model=GenerativeModel("gemini-2.0-flash-lite"),
        )
    ],
).evaluate(
    # Specify a candidate model for pairwise comparison
    model=model,
    prompt_template="# System_prompt\n{system_prompt} # Question\n{question}",
)

In [ ]:
notebook_utils.display_eval_result(
    pairwise_result,
    title=(
        "Gemini-2.0-Flash vs. Gemini-2.0-Flash-Lite SxS Pairwise Evaluation"
        " Results"
    ),
)

## Prompt Engineering

The *Vertex AI Gen AI Evaluation Service SDK* simplifies prompt engineering by streamlining the process of creating and evaluating multiple prompt templates. It allows you to efficiently test different prompts against a chosen dataset and compare their performance using comprehensive evaluation metrics. This empowers you to identify the most effective prompts for your specific use case and optimize your generative AI applications.

###  Compare and optimize prompt template design

#### Define an evaluation dataset


To perform pointwise inference, the evaluation dataset is required to contain the following fields:

* Instruction: Part of the input user prompt. It refers to the inference instruction that is sent to your LLM.
* Context: User input for the Gen AI model or application in the current turn.
* Reference: The ground truth to compare your LLM response to.

Your dataset must include a minimum of one evaluation example. We recommend around 100 examples to ensure high-quality aggregated metrics and statistically significant results.

In [ ]:
instruction = "Summarize the following article: \n"

context = [
    (
        "Typhoon Phanfone has killed at least one person, a US airman on"
        " Okinawa who was washed away by high waves. Thousands of households"
        " have lost power and Japan's two largest airlines have suspended many"
        " flights. The storm also forced the suspension of the search for"
        " people missing after last week's volcanic eruption. The"
        " storm-tracking website Tropical Storm Risk forecasts that Phanfone"
        " will rapidly lose power over the next few hours as it goes further"
        " into the Pacific Ocean. Typhoon Phanfone was downgraded from an"
        " earlier status of a super typhoon, but the Japan Meteorological"
        " Agency had warned it was still a dangerous storm. Japan averages 11"
        " typhoons a year, according to its weather agency. The typhoon made"
        " landfall on Monday morning near the central city of Hamamatsu, with"
        " winds of up to 180 km/h (112 mph). The airman was one of three US"
        " military personnel swept away by high waves whipped up by the typhoon"
        " off southern Okinawa island, where the US has a large military base."
        " The remaining two are still missing. A police spokesman said they had"
        " been taking photographs of the sea. A university student who was"
        " surfing off the seas of Kanagawa Prefecture, south of Tokyo, was also"
        " missing, national broadcast NHK reports. It said at least 10 people"
        " had been injured and 9,500 houses were without power. The storm was"
        " expected to deposit about 100mm of rain on Tokyo over 24 hours,"
        " according to the Transport Ministry website. Many schools were closed"
        " on Monday and two car companies in Japan halted production at some"
        " plants ahead of the storm. More than 174 domestic flights were"
        " affected nationwide, NHK state broadcaster said on Sunday. On Sunday,"
        " heavy rain delayed the Japanese Formula One Grand Prix in Suzaka."
        " French driver Jules Bianchi lost control in the wet conditions and"
        " crashed, sustaining a severe head injury."
    ),
    (
        "The blaze started at the detached building in Drivers End in Codicote,"
        " near Welwyn, during the morning. There was another fire at the"
        " building 20 years ago, after which fire-proof foil was placed under"
        " the thatch, which is protecting the main building. More than 15 fire"
        " engines and support vehicles were called to tackle the blaze. Roads"
        " in the area were closed and traffic diverted."
    ),
    (
        "The 18-year-old fell at the New Charter Academy on Broadoak Road in"
        " Ashton-under-Lyne at about 09:10 BST, Greater Manchester Police (GMP)"
        " said. GMP said he had gone to Manchester Royal Infirmary and his"
        ' condition was "serious". Principal Jenny Langley said the school'
        ' would remain "fully open" while police investigated. "Our thoughts'
        " are with the family and we're doing everything we can to support"
        ' them along with staff and pupils," she said.'
    ),
    (
        "But Belgian-born Dutchman Max Verstappen was unable to drive a car"
        " legally on his own in either country. That all changed on Wednesday"
        " when the youngster turned 18 and passed his driving test at the first"
        " attempt. Despite having competed in 14 grands prix since his debut in"
        " Australia in March, Verstappen admitted to feeling the pressure"
        ' during his test. "It\'s a relief," said the Toro Rosso driver, who'
        " finished ninth in Japan on Sunday and had only started driving"
        ' lessons a week ago. "I was a bit nervous to make mistakes, but the'
        ' exam went well." A bonus of turning 18 is that Verstappen will now be'
        " able to drink the champagne if he ever makes it onto the podium."
    ),
]

reference = [
    (
        "A powerful typhoon has brought many parts of Japan to a standstill and"
        " briefly battered Tokyo before heading out to sea."
    ),
    (
        "A major fire has been burning in the thatched roof of a large property"
        " in Hertfordshire."
    ),
    (
        "A student has been taken to hospital after falling from a balcony at a"
        " Greater Manchester school."
    ),
    (
        "He is Formula 1's youngest ever driver and in charge of a car that can"
        " reach over 200mph."
    ),
]

response = [
    (
        "Typhoon Phanfone, while downgraded from super typhoon status, caused"
        " significant disruption and tragedy in Japan. One US airman died after"
        " being swept away by high waves, with two more missing. The storm"
        " caused power outages for thousands, flight cancellations, and the"
        " suspension of rescue efforts for missing volcano victims. Heavy rain"
        " and strong winds led to school and factory closures, transportation"
        " disruptions, and at least 10 injuries. The typhoon is expected to"
        " weaken as it moves over the Pacific Ocean."
    ),
    (
        "A large fire broke out in a detached thatched building in Codicote,"
        " near Welwyn. This is the second fire at the building in 20 years."
        " Thankfully, fire-proof foil installed after the previous fire is"
        " protecting the main building. Over 15 fire engines and support"
        " vehicles responded, closing roads and diverting traffic in the area."
    ),
    (
        "An 18-year-old student at New Charter Academy in Ashton-under-Lyne"
        " suffered a serious fall and was hospitalized. The incident is under"
        " investigation by Greater Manchester Police, but the school remains"
        " open. The principal expressed support for the student's family and"
        " the school community."
    ),
    (
        "Max Verstappen, a Formula One driver, was finally able to get his"
        " driver's license at age 18. Despite already competing in 14 Grand"
        " Prix races, he was not of legal driving age in his native countries."
        " He admitted to being nervous but passed the test on his first"
        " attempt.  As an added bonus of turning 18, Verstappen can now enjoy"
        " champagne on the podium if he places."
    ),
]

eval_dataset = pd.DataFrame({
    "instruction": instruction,
    "context": context,
    "reference": reference,
})

#### Define an EvalTask

In [ ]:
EXPERIMENT_NAME = "eval-sdk-prompt-engineering"  # @param {type:"string"}

summarization_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[
        "rouge_l_sum",
        "bleu",
        "fluency",
        "coherence",
        "safety",
        "groundedness",
        "summarization_quality",
        "verbosity",
        "instruction_following",
        "text_quality",
    ],
    experiment=EXPERIMENT_NAME,
)

#### Run Evaluation

In [ ]:
# Define prompt templates to compare
prompt_templates = [
    (
        "Instruction: {instruction} such that you're explaining it to a 5 year"
        " old. Article: {context}. Summary:"
    ),
    "Article: {context}. Complete this task: {instruction}. Summary:",
    (
        "Goal: {instruction} and give me a TL;DR in five words. Here's an"
        " article: {context}. Summary:"
    ),
    (
        "Article: {context}. Reference Summary: {reference}. {instruction} to"
        " be more concise and verbose than the reference."
    ),
]

eval_results = []
for i, prompt_template in enumerate(prompt_templates):
  eval_result = summarization_eval_task.evaluate(
      prompt_template=prompt_template,
      model=GenerativeModel(
          "gemini-2.0-flash",
          generation_config={
              "temperature": 0.3,
              "max_output_tokens": 256,
              "top_k": 1,
          },
      ),
      # Customize eval service rate limit to improve speed.
      # See more details in https://cloud.google.com/vertex-ai/generative-ai/docs/models/run-evaluation#increase-quota
      evaluation_service_qps=5,
  )

  eval_results.append((f"Prompt Template #{i + 1}", eval_result))

#### Display Evaluation report and explanations

In [ ]:
for title, eval_result in eval_results:
  notebook_utils.display_eval_result(title=title, eval_result=eval_result)

In [ ]:
for title, eval_result in eval_results:
  notebook_utils.display_explanations(
      eval_result, metrics=["summarization_quality"], num=2
  )

#### Visualize Results

In [ ]:
notebook_utils.display_radar_plot(
    eval_results,
    metrics=["instruction_following", "fluency", "coherence", "text_quality"],
)

In [ ]:
notebook_utils.display_bar_plot(
    eval_results,
    metrics=["instruction_following", "fluency", "coherence", "text_quality"],
)

####  View Experiment log for evaluation runs

In [ ]:
summarization_eval_task.display_runs()

## Cleaning up

In [ ]:
delete_experiment = True

# Please set your LOCATION to the same one used during Vertex AI SDK initialization.
LOCATION = "us-central1"  # @param {type:"string"}

if delete_experiment:
  from google.cloud import aiplatform

  aiplatform.init(project=PROJECT_ID, location=LOCATION)
  experiment = aiplatform.Experiment(EXPERIMENT_NAME)
  experiment.delete()